# **Comparación y Selección de Modelos Predictivos**



En este capítulo se evalúa el desempeño de distintos enfoques de modelado para la predicción del precio de vehículos usados. Se comparan modelos de regresión lineal múltiple, XGBoost y redes neuronales, analizando sus resultados en términos de capacidad predictiva, complejidad y generalización. El objetivo es identificar el modelo que ofrece el mejor equilibrio entre precisión y robustez, considerando la naturaleza y el volumen del conjunto de datos.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/dataset/vehicles.parquet')

## **Partición del conjunto de datos**

In [ ]:
X = df.drop(columns=['price'])
y = df.price.values.reshape(-1,1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size=0.2,random_state=42)

In [ ]:
X_train.shape[0],X_test.shape[0],X_val.shape[0]

(118835, 37136, 29709)

Para el entrenamiento y evaluación de los modelos, el conjunto de datos fue dividido en subconjuntos de entrenamiento, validación y prueba, con tamaños de 118,835, 37,136 y 29,709 observaciones respectivamente. Esta partición permite entrenar los modelos, ajustar hiperparámetros y evaluar su capacidad de generalización sobre datos no vistos.

## **Preprocesamiento de datos**

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,PowerTransformer

In [ ]:
num = X_train.select_dtypes(include=['int64','float64']).columns
cat = X_train.select_dtypes(include=['object']).columns

In [ ]:
preprocessor = ColumnTransformer([
    ('power',PowerTransformer(),num),
    ('onehot',OneHotEncoder(handle_unknown='ignore'),cat)

])

tree_based_preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'),cat)
    ],
    remainder='passthrough'
)

El preprocesador definido en esta sección está orientado a modelos sensibles a la escala de las variables, como la regresión lineal múltiple y las redes neuronales artificiales. En estos casos, se aplica un PowerTransformer a las variables numéricas con el objetivo de reducir la asimetría y estabilizar la varianza, tal como se identificó durante el análisis exploratorio de los datos.

Para los algoritmos pertenecientes a la familia de los árboles de decisión, como XGBoost, se utiliza un preprocesador alternativo sin transformación de escala, dado que estos modelos operan mediante particiones matemáticas del espacio de características y no son sensibles a la escala de las variables. Esta diferenciación permite aplicar transformaciones adecuadas según la naturaleza de cada algoritmo y evita introducir complejidad innecesaria en los modelos basados en árboles.

## **Modelos**

### **Regresión Lineal Múltiple**

La regresión lineal simple consiste en encontrar la mejor línea recta que describa la relación entre una variable independiente y una variable dependiente, minimizando una función de pérdida, generalmente el error cuadrático medio.

En este proyecto se emplea una regresión lineal múltiple, ya que el precio del vehículo se modela a partir de múltiples variables explicativas, tales como el año del vehículo, el kilometraje, la condición, el tipo de transmisión, el sistema de tracción y la marca, entre otras. Este enfoque permite capturar el efecto conjunto de varias características sobre el precio final.


In [ ]:
X_train.columns

Index(['year', 'manufacturer', 'condition', 'cylinders', 'fuel', 'odometer',
       'title_status', 'transmission', 'drive', 'type', 'paint_color'],
      dtype='object')

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

In [ ]:
lm = Pipeline([
    ('preprocesor',preprocessor),
    ('model',LinearRegression())
])

In [ ]:
lm.fit(X_train,y_train)

Pipeline(steps=[('preprocesor',
                 ColumnTransformer(transformers=[('power', PowerTransformer(),
                                                  Index(['year', 'odometer'], dtype='object')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['manufacturer', 'condition', 'cylinders', 'fuel', 'title_status',
       'transmission', 'drive', 'type', 'paint_color'],
      dtype='object'))])),
                ('model', LinearRegression())])

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
lm.score(X_train,y_train)

0.7106965489798269

In [ ]:
lm.score(X_test,y_test)

0.7088499371341006

In [ ]:
lm.score(X_val,y_val)

0.7120776754473721

In [ ]:
def mae(modelo):
  y_pred_train = modelo.predict(X_train)
  y_pred_test = modelo.predict(X_test)
  y_pred_val = modelo.predict(X_val)

  mae_train = np.round(mean_absolute_error(y_train,y_pred_train),2)
  mae_test = np.round(mean_absolute_error(y_test,y_pred_test),2)
  mae_val = np.round(mean_absolute_error(y_val,y_pred_val),2)


  print(f"MAE Train : {mae_train}")
  print(f"MAE Test: {mae_test}")
  print(f"MAE Val: {mae_test}")


In [ ]:
mae(lm)

MAE Train : 3785.59
MAE Test: 3796.7
MAE Val: 3796.7


Como primer enfoque se implementó un modelo de Regresión Lineal Múltiple, el cual sirve como modelo base (baseline) para evaluar el desempeño de algoritmos más complejos.

El modelo obtuvo un R² score cercano al 0.70, lo que indica que es capaz de explicar aproximadamente el 70% de la variabilidad del precio a partir de las variables independientes utilizadas. Este resultado es bastante sólido considerando que se trata de un algoritmo lineal y relativamente simple.

En cuanto a las métricas de error absoluto medio (MAE), se obtuvieron los siguientes resultados:


La cercanía entre los errores de entrenamiento, prueba y validación sugiere que el modelo no presenta sobreajuste, mostrando un comportamiento estable y consistente al generalizar sobre datos no vistos.

En conjunto, estos resultados confirman que la regresión lineal múltiple ofrece un buen punto de partida, estableciendo una referencia clara para comparar el desempeño de modelos más avanzados como XGBoost y Redes Neuronales Artificiales.

## **XGBoost**


XGBoost es un algoritmo de aprendizaje basado en árboles de decisión que pertenece a la familia de los métodos de gradient boosting. A diferencia de los bosques aleatorios, donde los árboles se entrenan de forma independiente y sus resultados se promedian, en XGBoost los árboles se construyen de manera secuencial.

Cada nuevo árbol se entrena para corregir los errores cometidos por los árboles anteriores, optimizando una función de pérdida mediante descenso por gradiente. El parámetro learning rate controla cuánto contribuye cada árbol al modelo final, permitiendo un aprendizaje gradual y evitando el sobreajuste.

El resultado final del modelo es la suma ponderada de las predicciones de todos los árboles, lo que permite capturar relaciones complejas y no lineales en los datos, logrando un alto desempeño tanto en tareas de regresión como de clasificación.

In [ ]:
from xgboost import XGBRegressor

In [ ]:
xgb = Pipeline([
    ('preprocessor',tree_based_preprocessor),
    ('model',XGBRegressor(max_depth=8,
                          n_estimators=1000,
                          learning_rate = 0.01,
                          random_state=42,
                          subsample = 0.8,
                          colsample_by_tree = 0.7))])


#### **Hiperparámetros del modelo XGBoost**

* **max_depth:**
Define la profundidad máxima de cada árbol de decisión. Un valor relativamente alto permite capturar relaciones complejas y no lineales entre las variables, aunque se controla el riesgo de sobreajuste mediante otros hiperparámetros de regularización.

* **n_estimators:**
Indica el número total de árboles que se entrenan de forma secuencial. Al utilizar una gran cantidad de árboles junto con un learning rate bajo, el modelo aprende de manera gradual, logrando una mejor generalización.

* **learning_rate:**
Controla la contribución de cada árbol al modelo final. Un valor bajo reduce el impacto de cada árbol individual, permitiendo un aprendizaje más estable y disminuyendo el riesgo de sobreajuste.

* **random_state:**
Fija la semilla aleatoria para garantizar la reproducibilidad de los resultados en diferentes ejecuciones del modelo.

* **subsample:**
Especifica la proporción de observaciones utilizadas para entrenar cada árbol. Al usar solo el 80% de los datos en cada iteración, se introduce aleatoriedad que ayuda a mejorar la capacidad de generalización del modelo.

* **colsample_bytree:**
Determina el porcentaje de variables seleccionadas aleatoriamente para construir cada árbol. Esto reduce la dependencia entre árboles y ayuda a mitigar el sobreajuste, especialmente en datasets con muchas variables.

In [ ]:
xgb.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['manufacturer', 'condition', 'cylinders', 'fuel', 'title_status',
       'transmission', 'drive', 'type', 'paint_color'],
      dtype='object'))])),
                ('model',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_by_tree...
                              feature_types=None, feature_weights=None,
                              gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.01,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=8, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=1000, n_jobs=None, ...))])

In [ ]:
xgb.score(X_train,y_train)

0.8241881132125854

In [ ]:
xgb.score(X_test,y_test)

0.8035611510276794

In [ ]:
xgb.score(X_val,y_val)

0.8051928281784058

In [ ]:
mae(xgb)

MAE Train : 2752.0
MAE Test: 2902.53
MAE Val: 2902.53


Como segundo enfoque se implementó un modelo XGBoost, perteneciente a la familia de los árboles de decisión potenciados (gradient boosting), el cual es capaz de modelar relaciones no lineales complejas y capturar interacciones entre variables que un modelo lineal no puede representar.

El modelo obtuvo un R² score cercano al 0.80, lo que representa una mejora considerable respecto a la regresión lineal múltiple (R² ≈ 0.70). Este incremento indica que XGBoost logra explicar aproximadamente el 80% de la variabilidad del precio de los vehículos, reflejando un ajuste más preciso al comportamiento real de los datos.

En cuanto al Error Absoluto Medio (MAE), considerando que la variable objetivo se encuentra en escala de dólares estos valores indican que, en promedio, el modelo se equivoca alrededor de **$ 2,900$** dólares al predecir el precio de un vehículo, lo cual representa una reducción significativa del error en comparación con la regresión lineal múltiple **(≈ $3,800).**



La diferencia moderada entre el MAE de entrenamiento y el de test/validación sugiere que el modelo generaliza adecuadamente, sin evidencias claras de sobreajuste, especialmente considerando el gran tamaño del dataset (≈185,000 registros).

En conclusión, XGBoost demuestra un mejor equilibrio entre sesgo y varianza, consolidándose como un modelo robusto y altamente competitivo para la predicción de precios de vehículos dentro de este proyecto.

## **Redes Neuronales**

Las redes neuronales artificiales son algoritmos bioinspirados en el funcionamiento de las neuronas del cerebro humano. Tienen la capacidad de aprender patrones complejos y ajustar sus parámetros de forma iterativa mediante la minimización de una función de pérdida, corrigiendo sus errores a lo largo del proceso de entrenamiento.

Gracias a su arquitectura, las redes neuronales son especialmente eficaces para modelar relaciones no lineales presentes en los datos. No obstante, otros algoritmos como XGBoost también pueden capturar este tipo de relaciones de manera eficiente mediante estructuras basadas en árboles de decisión.

A pesar de su buen desempeño predictivo, las redes neuronales presentan una limitación importante en términos de interpretabilidad, ya que no es posible conocer con exactitud cómo cada variable contribuye a la predicción final. Por esta razón, desde una perspectiva estadística, suelen ser consideradas modelos de “caja negra”, lo que puede representar un desafío en contextos donde la explicación del modelo es tan importante como su precisión.

In [ ]:
scaler = preprocessor.fit(X_train)

In [ ]:
X_train_scaler = scaler.transform(X_train)
X_test_scaler = scaler.transform(X_test)
X_val_scaler = scaler.transform(X_val)

In [ ]:
def array(x):
  return x.toarray()

In [ ]:
X_train_scaler = array(X_train_scaler)
X_test_scaler = array(X_test_scaler)
X_val_scaler = array(X_val_scaler)

In [ ]:
y_scaler = PowerTransformer().fit(y_train)

In [ ]:
y_train_scaler = y_scaler.transform(y_train)
y_test_scaler = y_scaler.transform(y_test)
y_val_scaler = y_scaler.transform(y_val)

Las redes neuronales artificiales requieren un preprocesamiento más riguroso de los datos, ya que son altamente sensibles a la escala de las variables. Una adecuada normalización o transformación de las variables numéricas es fundamental para que el proceso de optimización converja de manera eficiente.

En ausencia de este preprocesamiento, la red puede presentar convergencia lenta, inestabilidad durante el entrenamiento o quedar atrapada en mínimos subóptimos, lo que incrementa significativamente el tiempo de entrenamiento y afecta el desempeño del modelo. Por esta razón, se aplican transformaciones como el PowerTransformer, garantizando que las variables se encuentren en escalas comparables y facilitando el aprendizaje del modelo.

In [ ]:
import torch
from torch.utils.data import Dataset,DataLoader

In [ ]:
class CarPriceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).float()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
train_dataset = CarPriceDataset(X_train_scaler, y_train_scaler)
val_dataset   = CarPriceDataset(X_val_scaler, y_val_scaler)
test_dataset  = CarPriceDataset(X_test_scaler, y_test_scaler)

In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=0
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=0
)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=0
)


Para entrenar modelos en PyTorch es necesario encapsular los datos dentro de una clase que herede de `torch.utils.data.Dataset`.  
En este caso se define `CarPriceDataset`, cuyo objetivo es adaptar los datos previamente preprocesados (arrays de NumPy) al formato requerido por PyTorch.

La clase convierte las variables de entrada y el objetivo (`X`, `y`) a tensores de tipo `float`, permitiendo:
- Acceso indexado a los datos mediante `__getitem__`
- Cálculo automático del tamaño del conjunto con `__len__`
- Integración directa con `DataLoader` para entrenamiento por mini-lotes, barajado y paralelización

Este enfoque sigue las buenas prácticas de PyTorch y permite un entrenamiento eficiente y escalable de modelos de redes neuronales.


### **Arquitectura de la red neuronal**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
X_train_scaler.shape[1]

95

In [ ]:
class Model(nn.Module):
  def __init__(self):
    super().__init__()

    self.fc1 = nn.Linear(95,128)
    self.bn1 = nn.BatchNorm1d(128)

    self.fc2 = nn.Linear(128,64)
    self.bn2 = nn.BatchNorm1d(64)


    self.fc3 = nn.Linear(64,32)
    self.bn3 = nn.BatchNorm1d(32)

    self.dropout = nn.Dropout(0.3)
    self.fc4 = nn.Linear(32,1)

  def forward(self,x):

    x = self.fc1(x)
    x = self.bn1(x)
    x = F.relu(x)

    x = self.fc2(x)
    x = self.bn2(x)
    x = F.relu(x)

    x = self.fc3(x)
    x = self.bn3(x)
    x = F.relu(x)


    x= self.dropout(x)
    x = self.fc4(x)

    return x

In [ ]:
model = Model()

In [ ]:
model

Model(
  (fc1): Linear(in_features=95, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (bn3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc4): Linear(in_features=32, out_features=1, bias=True)
)

El modelo recibe como entrada un vector de 95 características, previamente preprocesadas y normalizadas.

La primera capa oculta está compuesta por 128 neuronas con función de activación ReLU, cuya finalidad es capturar relaciones no lineales complejas presentes en los datos de entrada. Esta capa permite una transformación inicial rica del espacio de características.

Posteriormente, el modelo incorpora una segunda capa densa de 64 neuronas, también con activación ReLU, que reduce progresivamente la dimensionalidad y refina las representaciones aprendidas, favoreciendo una mejor generalización del modelo.

La tercera capa oculta cuenta con 32 neuronas y mantiene la activación ReLU, enfocándose en la extracción de características de mayor nivel y en la consolidación de patrones relevantes para la tarea de predicción.

Con el objetivo de mitigar el sobreajuste, se añade una capa de Dropout con una tasa de 0.5, lo que implica que durante el entrenamiento se desactiva aleatoriamente el 50% de las neuronas de la capa anterior, promoviendo un aprendizaje más robusto y estable.

Finalmente, la red concluye con una capa densa de una sola neurona y función de activación lineal, adecuada para la generación de valores continuos, correspondiente a la salida del modelo de regresión.

En conjunto, esta arquitectura permite modelar relaciones no lineales complejas en datos tabulares, manteniendo un equilibrio entre capacidad predictiva y regularización.

In [ ]:
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

Para el entrenamiento de la red neuronal se seleccionó la función de pérdida **MAE (Mean Absolute Error)**, implementada en PyTorch como `nn.L1Loss()`.  
Esta métrica mide el error promedio en términos absolutos entre el precio real y el precio predicho, lo cual resulta especialmente adecuado en el contexto del mercado automotriz, donde el precio se expresa en dólares y los errores son fácilmente interpretables.

A diferencia de métricas como MSE, la MAE es **menos sensible a valores atípicos**, una característica relevante considerando que el dataset incluye vehículos de gama alta con precios significativamente mayores al promedio.

Como algoritmo de optimización se utilizó **Adam**, con una tasa de aprendizaje de `1e-3`.  
Adam combina las ventajas de métodos adaptativos como RMSProp y Momentum, permitiendo:
- Ajustes dinámicos de la tasa de aprendizaje por parámetro
- Convergencia más rápida en problemas de alta dimensionalidad
- Mayor estabilidad durante el entrenamiento de redes profundas

Esta combinación de MAE como función de pérdida y Adam como optimizador proporciona un balance adecuado entre **robustez**, **velocidad de convergencia** y **estabilidad del entrenamiento**.


### **Entrenamiento de la red neuronal**

In [ ]:
def train_one_epoch(model, dataloader, optimizer, criterion, device="cpu"):
    model.train()
    running_loss = 0.0

    for X_batch, y_batch in dataloader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()

        preds = model(X_batch)
        loss = criterion(preds, y_batch)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    return running_loss / len(dataloader)


La función `train_one_epoch` encapsula el proceso de entrenamiento de una época completa.  
El modelo se coloca en modo entrenamiento mediante `model.train()`, lo cual es esencial para activar correctamente capas como Dropout.

Durante cada iteración:
- Se transfieren los datos al dispositivo de cómputo (CPU o GPU)
- Se realiza la propagación hacia adelante
- Se calcula la función de pérdida
- Se ejecuta la retropropagación del error
- Se actualizan los pesos del modelo usando el optimizador

El error acumulado se pondera por el tamaño de cada lote y se normaliza al final de la época,
obteniendo una estimación representativa del error medio sobre el conjunto de entrenamiento.


In [ ]:
def evaluate(model, dataloader, criterion, device="cpu"):
    model.eval()
    running_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            preds = model(X_batch)
            loss = criterion(preds, y_batch)

            running_loss += loss.item()

    return running_loss / len(test_loader)

La función `evaluate` se utiliza para medir el desempeño del modelo en los conjuntos de validación o prueba.  
El modelo se coloca en modo evaluación con `model.eval()`, lo cual desactiva comportamientos estocásticos como Dropout,
garantizando resultados consistentes.

El uso de `torch.no_grad()` reduce el consumo de memoria y acelera la inferencia,
ya que no se calculan gradientes durante la evaluación.

Al igual que en el entrenamiento, la pérdida se acumula y se normaliza respecto al número total de observaciones,
permitiendo comparar directamente los resultados entre entrenamiento, validación y prueba.


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

Model(
  (fc1): Linear(in_features=95, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (bn3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc4): Linear(in_features=32, out_features=1, bias=True)
)

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0, path='best_model.pt'):
        self.patience = patience
        self.min_delta = min_delta
        self.path = path
        self.counter = 0
        self.best_loss = np.inf
        self.early_stop = False

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            torch.save(model.state_dict(), self.path)
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

A diferencia de TensorFlow/Keras, PyTorch no incorpora un mecanismo de *Early Stopping* de forma nativa.  
Por esta razón, se implementó una clase personalizada cuyo objetivo es **detener el entrenamiento cuando el desempeño en el conjunto de validación deja de mejorar**, evitando así el sobreajuste y entrenamientos innecesariamente largos.

El criterio de parada se basa en el seguimiento de la pérdida de validación:
- Si la métrica monitoreada no mejora después de un número definido de épocas (`patience`)
- El entrenamiento se interrumpe automáticamente
- Se conservan los mejores pesos observados durante el proceso

Este enfoque permite replicar el comportamiento del `EarlyStopping` de Keras, manteniendo al mismo tiempo la flexibilidad y control que caracteriza a PyTorch.


Se implementó el callback EarlyStopping para monitorear la función de pérdida sobre el conjunto de validación (val_loss).
El entrenamiento se detiene automáticamente si después de 5 épocas consecutivas no se observa una mejora en el desempeño del modelo, evitando así el sobreajuste y reduciendo el tiempo de entrenamiento innecesario.

Además, se activó la opción restore_best_weights=True, lo que garantiza que el modelo conserve los pesos correspondientes a la época con mejor desempeño en los datos de validación.

In [ ]:
epochs = 30
early_stopping = EarlyStopping(patience=5)

for epoch in range(epochs):
    train_loss = train_one_epoch(
        model, train_loader, optimizer, criterion, device
    )

    val_loss = evaluate(
        model, test_loader, criterion, device
    )

    print(
        f"Epoch {epoch+1:02d} | "
        f"Train MAE: {train_loss:.2f} | "
        f"Val MAE: {val_loss:.2f}"
    )


    early_stopping(val_loss, model)

    if early_stopping.early_stop:
        print("Early stopping")
        break

Epoch 01 | Train MAE: 0.40 | Val MAE: 0.33
Epoch 02 | Train MAE: 0.38 | Val MAE: 0.33
Epoch 03 | Train MAE: 0.38 | Val MAE: 0.33
Epoch 04 | Train MAE: 0.38 | Val MAE: 0.32
Epoch 05 | Train MAE: 0.37 | Val MAE: 0.34
Epoch 06 | Train MAE: 0.37 | Val MAE: 0.32
Epoch 07 | Train MAE: 0.37 | Val MAE: 0.32
Epoch 08 | Train MAE: 0.37 | Val MAE: 0.32
Epoch 09 | Train MAE: 0.37 | Val MAE: 0.32
Epoch 10 | Train MAE: 0.37 | Val MAE: 0.32
Epoch 11 | Train MAE: 0.37 | Val MAE: 0.31
Epoch 12 | Train MAE: 0.37 | Val MAE: 0.31
Epoch 13 | Train MAE: 0.36 | Val MAE: 0.31
Epoch 14 | Train MAE: 0.36 | Val MAE: 0.31
Epoch 15 | Train MAE: 0.36 | Val MAE: 0.31
Epoch 16 | Train MAE: 0.36 | Val MAE: 0.32
Epoch 17 | Train MAE: 0.36 | Val MAE: 0.31
Early stopping


El entrenamiento del modelo se realiza mediante un ciclo de épocas controlado por un mecanismo de *Early Stopping*.  
Inicialmente se define un número máximo de épocas (`epochs = 30`), así como un criterio de parada temprana con una paciencia de 10 épocas consecutivas sin mejora en la métrica de validación.

En cada época se ejecutan los siguientes pasos:
1. Se entrena el modelo sobre el conjunto de entrenamiento utilizando mini-lotes, calculando el **MAE promedio** de la época.
2. Se evalúa el desempeño del modelo sobre el conjunto de validación, sin actualizar los pesos.
3. Se muestran las métricas de entrenamiento y validación para monitorear el proceso de aprendizaje.
4. Se verifica el criterio de *Early Stopping*, comparando la pérdida de validación actual con el mejor valor observado.

Si la pérdida de validación no mejora durante el número de épocas definido por la paciencia,
el entrenamiento se detiene automáticamente y se restauran los mejores pesos del modelo.


In [ ]:
X_train_scaler = torch.from_numpy(X_train_scaler).float()
X_test_scaler = torch.from_numpy(X_test_scaler).float()
X_val_scaler = torch.from_numpy(X_val_scaler).float()

In [ ]:
y_train_pred = model(X_train_scaler)
y_test_pred = model(X_test_scaler)
y_val_pred = model(X_val_scaler)

In [ ]:
y_train_pred = y_train_pred.detach().numpy()
y_test_pred = y_test_pred.detach().numpy()
y_val_pred = y_val_pred.detach().numpy()

In [ ]:
y_train_pred = y_scaler.inverse_transform(y_train_pred)
y_test_pred = y_scaler.inverse_transform(y_test_pred)
y_val_pred = y_scaler.inverse_transform(y_val_pred)

In [ ]:
print(f"MAE Train: {mean_absolute_error(y_train,y_train_pred)}")
print(f"MAE Test: {mean_absolute_error(y_test,y_test_pred)}")
print(f"MAE Val: {mean_absolute_error(y_val,y_val_pred)}")

MAE Train: 2756.39599609375
MAE Test: 2838.15625
MAE Val: 2846.576416015625


In [ ]:
from sklearn.metrics import r2_score

In [ ]:
print(f"R2 Train: {r2_score(y_train,y_train_pred)}")
print(f"R2 Test: {r2_score(y_test,y_test_pred)}")
print(f"R2 Val: {r2_score(y_val,y_val_pred)}")

R2 Train: 0.8058374524116516
R2 Test: 0.7988194823265076
R2 Val: 0.7986976504325867


En términos de Error Absoluto Medio (MAE), considerando que la variable objetivo se encuentra en escala de dólares, se obtuvieron los siguientes resultados:


Estos valores indican que, en promedio, la red neuronal presenta un error de predicción cercano a $2,800 dólares, logrando una ligera mejora respecto a XGBoost y una reducción notable frente a la regresión lineal múltiple.

La cercanía entre los errores de entrenamiento, validación y prueba sugiere que la red neuronal generaliza adecuadamente, sin señales claras de sobreajuste. Además, los tiempos de inferencia observados durante la evaluación reflejan una convergencia estable, favorecida por un preprocesamiento más estricto de los datos (escalado y transformación de distribuciones).

En conclusión, la red neuronal demuestra una alta capacidad para modelar relaciones no lineales complejas, aunque su naturaleza de “caja negra” limita la interpretabilidad frente a modelos basados en árboles. Aun así, su desempeño la convierte en una alternativa sólida y competitiva para la predicción del precio de vehículos.

## **Elección modelo**

Los resultados obtenidos muestran que la red neuronal presenta el mejor desempeño en términos de MAE,
con errores de aproximadamente **2756 USD en test y 2846 USD en validación**.
Sin embargo, el modelo **XGBoost** alcanza un desempeño competitivo, con un MAE cercano a **2,900 USD**,
una diferencia relativamente pequeña considerando la escala del problema y el rango de precios de los vehículos.

A pesar de que la red neuronal logra una ligera mejora en precisión, se opta por **XGBoost como modelo candidato para implementación**,
debido a varios factores clave:

- **Menor tiempo de entrenamiento**, incluso con un volumen de datos elevado  
- **Menores requerimientos de preprocesamiento**, al no ser sensible a la escala de las variables  
- **Mayor interpretabilidad**, permitiendo analizar la importancia de las variables y comprender mejor las decisiones del modelo  
- **Mayor control del sobreajuste**, gracias al ajuste explícito de hiperparámetros como `max_depth`, `min_child_weight`, `subsample` y `colsample_bytree`

Desde una perspectiva de negocio e ingeniería, la diferencia de error entre ambos modelos no justifica el aumento en complejidad,
costo computacional y dificultad de interpretación que implica el uso de una red neuronal.


### **Ajuste de hiperprametros**

In [ ]:
param_grid = {

    'model__min_child_weight': [3, 5, 7],
    'model__reg_lambda': [0.5, 1.0, 1.5],
    'model__gamma': [0.1,0.2,0.3]
}


El parámetro `min_child_weight` define el **peso mínimo total de observaciones** que debe tener un nodo hijo para que una partición sea válida.

Valores más altos hacen que el modelo sea más conservador, ya que:
- Evitan divisiones basadas en muy pocos datos
- Reducen el riesgo de sobreajuste
- Favorecen árboles más simples y generalizables

En datasets grandes como el presente, este parámetro resulta especialmente útil para
controlar particiones demasiado específicas que podrían ajustarse al ruido.


`reg_lambda` controla la regularización **L2** aplicada a los pesos de las hojas del árbol.

Un valor mayor:
- Penaliza pesos grandes
- Suaviza las predicciones del modelo
- Reduce la varianza

Este tipo de regularización es efectiva para estabilizar el modelo y evitar
que se ajuste en exceso a patrones particulares del conjunto de entrenamiento.


El parámetro `gamma` representa la **ganancia mínima requerida** para realizar una división adicional en un nodo.

Valores más altos de `gamma`:
- Hacen que el modelo sea más conservador
- Evitan divisiones con mejoras marginales
- Generan árboles menos profundos y más robustos

Este parámetro actúa como un filtro que asegura que cada nueva división
aporta una mejora significativa al modelo.


In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
grid = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring='neg_mean_absolute_error',          # o 'neg_root_mean_squared_error'
    cv=3,
    n_jobs=-1,
    verbose=2
)

In [ ]:
grid.fit(X_train,y_train)

In [ ]:
grid.best_params_

{'model__gamma': 0.1, 'model__min_child_weight': 3, 'model__reg_lambda': 0.5}

In [ ]:
xgb = Pipeline([
    ('preprocessor',tree_based_preprocessor),
    ('model',XGBRegressor(max_depth=8,
                          n_estimators=1000,
                          learning_rate = 0.01,
                          random_state=42,
                          subsample = 0.8,
                          colsample_by_tree = 0.7,
                          gamma = 0.1,
                          min_child_weight=3,
                          reg_lambda = 0.5))])

In [ ]:
xgb.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['manufacturer', 'condition', 'cylinders', 'fuel', 'title_status',
       'transmission', 'drive', 'type', 'paint_color'],
      dtype='object'))])),
                ('model',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_by_tree...
                              feature_types=None, feature_weights=None,
                              gamma=0.1, grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=0.01,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=8, max_leaves=None, min_child_weight=3,
                              missing=nan, monotone_constraints=None,
                              multi_strategy=None, n_estimators=1000,
                              n_jobs=None, ...))])

In [ ]:
mae(xgb)

MAE Train : 2746.81
MAE Test: 2896.85
MAE Val: 2896.85


In [ ]:
xgb.score(X_train,y_train)

0.8244479298591614

In [ ]:
xgb.score(X_val,y_val)

0.8057473301887512

In [ ]:
xgb.score(X_test,y_test)

0.8042787909507751

## **Guardar modelo**

In [ ]:
import pandas as pd

data_test = [
    {
        'year': 2019,
        'manufacturer': 'toyota',
        'condition': 'good',
        'cylinders': '4_or_less',
        'fuel': 'gas',
        'odometer': 68000,
        'title_status': 'clean',
        'transmission': 'automatic',
        'drive': 'fwd',
        'type': 'sedan',
        'paint_color': 'silver'
    },
    {
        'year': 2016,
        'manufacturer': 'ford',
        'condition': 'excellent',
        'cylinders': '8_or_more',
        'fuel': 'gas',
        'odometer': 85000,
        'title_status': 'clean',
        'transmission': 'automatic',
        'drive': '4wd',
        'type': 'pickup',
        'paint_color': 'black'
    },

    {
        'year': 2014,
        'manufacturer': 'jeep',
        'condition': 'good',
        'cylinders': '5_6_cyl',
        'fuel': 'gas',
        'odometer': 120000,
        'title_status': 'clean',
        'transmission': 'automatic',
        'drive': '4wd',
        'type': 'SUV',
        'paint_color': 'green'
    },
    {
        'year': 2018,
        'manufacturer': 'bmw',
        'condition': 'excellent',
        'cylinders': '5_6_cyl',
        'fuel': 'gas',
        'odometer': 55000,
        'title_status': 'clean',
        'transmission': 'automatic',
        'drive': 'rwd',
        'type': 'sedan',
        'paint_color': 'blue'
    }
]

new_data = pd.DataFrame(data_test)



In [ ]:
new_data['predict'] = xgb.predict(new_data)

In [ ]:
new_data

,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color,predict
0,2019,toyota,good,4_or_less,gas,68000,clean,automatic,fwd,sedan,silver,16385.753906
1,2016,ford,excellent,8_or_more,gas,85000,clean,automatic,4wd,pickup,black,31017.648438
2,2014,jeep,good,5_6_cyl,gas,120000,clean,automatic,4wd,SUV,green,17216.335938
3,2018,bmw,excellent,5_6_cyl,gas,55000,clean,automatic,rwd,sedan,blue,29473.931641


In [ ]:
import joblib

joblib.dump(xgb,'model.pkl')

['model.pkl']

Esto nos permite persistir el modelo en disco para reutilizarlo posteriormente sin necesidad de volver a entrenarlo.

Entrenar un modelo puede tomar tiempo y recursos computacionales, por lo que guardar el modelo es una buena práctica en proyectos de Machine Learning productivos.

## **Conclusión**



En este proyecto se evaluaron tres enfoques distintos para la estimación del precio de vehículos usados:

- Regresión Lineal Múltiple  
- XGBoost  
- Red Neuronal implementada en PyTorch  

Cada modelo fue analizado en términos de capacidad predictiva, costo computacional e interpretabilidad.

---

### 🔹 1. Regresión Lineal Múltiple

La regresión lineal múltiple obtuvo un **R² ≈ 0.70**, lo que indica que el modelo es capaz de explicar aproximadamente el 70% de la variabilidad del precio de los vehículos.

En términos de error:

- **MAE Train:** 3785.59  
- **MAE Test:** 3796.7  
- **MAE Val:** 3796.7  

Considerando que el precio está en dólares, el error promedio absoluto ronda los **$3,800 USD**.

#### Ventajas
- Modelo simple y estadísticamente sólido.
- Alta interpretabilidad.
- Bajo costo computacional.
- Fácil de explicar al cliente.

#### Limitaciones
- Dificultad para capturar relaciones no lineales complejas.

---

### 🔹 2. XGBoost

El modelo XGBoost mostró una mejora considerable frente a la regresión lineal, alcanzando un:

- **R² ≈ 0.80**
- **MAE Train:** 2752.0  
- **MAE Test:** 2902.53  
- **MAE Val:** 2902.53  

Esto representa una reducción significativa del error promedio a aproximadamente **$2,900 USD**.

Dado el tamaño del dataset (~185,000 registros) y la complejidad del mercado automotriz, fue necesario utilizar una profundidad de árbol relativamente alta. Para controlar el sobreajuste se ajustaron hiperparámetros como:

- `min_child_weight`
- `gamma`
- `reg_lambda`

#### Ventajas
- Excelente capacidad para modelar relaciones no lineales.
- Buen equilibrio entre rendimiento y tiempo de entrenamiento.
- Posibilidad de interpretar importancia de variables (Feature Importance, SHAP).
- Más explicable que una red neuronal.

---

### 🔹 3. Red Neuronal (PyTorch)

La red neuronal implementada en PyTorch mostró un desempeño sólido, alcanzando un:

- **R² ≈ 0.80**
- **MAE Train:** 2756.40  
- **MAE Test:** 2838.16  
- **MAE Val:** 2846.58  

Esto indica que el modelo es capaz de explicar aproximadamente el **80% de la variabilidad del precio**, lo cual representa una mejora considerable respecto a la regresión lineal múltiple y un desempeño competitivo frente a XGBoost.

En términos prácticos, el error promedio absoluto se sitúa alrededor de **$2,840 USD**, lo cual es significativamente menor que el obtenido con la regresión lineal y muy cercano al rendimiento de XGBoost.

---

#### Ventajas

- Capacidad para modelar relaciones no lineales complejas.
- Buen equilibrio entre sesgo y varianza.
- Arquitectura flexible y escalable.
- Posibilidad de incorporar regularización (Dropout, BatchNorm, Early Stopping).

---

#### Limitaciones

- Mayor costo computacional comparado con XGBoost.
- Sensible a escalas (requiere preprocesamiento más riguroso).
- Menor interpretabilidad (modelo tipo *caja negra*).
- Puede presentar variabilidad en resultados si no se controla la semilla.

---

### 🎯 Comparación General

- **Regresión Lineal Múltiple:** R² ≈ 0.70  
- **XGBoost:** R² ≈ 0.80  
- **Red Neuronal (PyTorch):** R² ≈ 0.80  

En conclusión, aunque la red neuronal logra un desempeño muy competitivo, XGBoost sigue siendo una opción extremadamente sólida debido a su equilibrio entre rendimiento, velocidad de entrenamiento e interpretabilidad.



---

## Conclusión Final

Aunque la red neuronal obtuvo el mejor desempeño numérico, **XGBoost representa la mejor solución balanceada para un entorno productivo**, considerando:

- Tiempo de entrenamiento menor.
- Buen desempeño predictivo.
- Mayor interpretabilidad.
- Facilidad para explicar resultado
